# MicroGPT 入門ノート（日本語版）

このノートブックは、わずか数百行で書かれた最小構成の GPT 実装を **初学者向け** に学ぶ教材です。
学習・推論を実行しながら、損失や確率分布を可視化して動作を理解します。

## 謝辞

この教材は Andrej Karpathy 氏の MicroGPT 実装をもとに、教育目的で日本語化・可視化を追加したものです。

- Original: https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95
- Author: @karpathy

## 1. 実行準備
Colab ではそのまま動きます。必要なら以下を実行してください。

In [ ]:
%pip -q install matplotlib

In [ ]:
import math
import random
from typing import List

import matplotlib.pyplot as plt

random.seed(42)

## 2. 日本語サンプルデータ（self-contained）
外部ファイルなしで学べるよう、簡単な日本語（ひらがな・カタカナ）サンプルを内蔵しています。

必要に応じて `docs` を増やしてください。

In [ ]:
docs = [
    "さくら", "すず", "はる", "ひなた", "みお",
    "ゆい", "あかり", "りん", "あおい", "めい",
    "たくみ", "そうた", "けん", "れん", "だいき",
    "しょう", "こうた", "はると", "ゆう", "なお",
    "サクラ", "ユイ", "アオイ", "レン", "ハルト",
    "しおり", "かな", "まな", "りお", "えま",
]
random.shuffle(docs)
print(f"文書数: {len(docs)}")
print("サンプル:", docs[:10])

## 3. トークナイザ（文字単位）
この最小実装では単語ではなく **1文字ずつ** をトークンとして扱います。

In [ ]:
uchars = sorted(set(''.join(docs)))
BOS = len(uchars)
vocab_size = len(uchars) + 1

stoi = {ch: i for i, ch in enumerate(uchars)}
itos = {i: ch for ch, i in stoi.items()}
itos[BOS] = "<BOS>"

print(f"語彙数(vocab_size): {vocab_size}")
print("語彙:", ''.join(uchars))

## 4. 自動微分 Value クラス
各値が『どの計算から生まれたか』を覚えておき、最後に `backward()` で勾配を逆伝播します。

In [ ]:
class Value:
    __slots__ = ('data', 'grad', '_children', '_local_grads')

    def __init__(self, data, children=(), local_grads=()):
        # 順伝播で得られる実数値
        self.data = data
        # 逆伝播で蓄積される勾配
        self.grad = 0
        # 計算グラフ上の親ノード
        self._children = children
        # 親に対する局所勾配
        self._local_grads = local_grads

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data, (self, other), (1, 1))

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data, (self, other), (other.data, self.data))

    def __pow__(self, other):
        return Value(self.data ** other, (self,), (other * self.data ** (other - 1),))

    def log(self):
        return Value(math.log(self.data), (self,), (1 / self.data,))

    def exp(self):
        return Value(math.exp(self.data), (self,), (math.exp(self.data),))

    def relu(self):
        return Value(max(0, self.data), (self,), (float(self.data > 0),))

    def __neg__(self): return self * -1
    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other ** -1
    def __rtruediv__(self, other): return other * self ** -1

    def backward(self):
        # 計算グラフをトポロジカル順序に並べる
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1

        # 逆順にたどって勾配を伝播
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad += local_grad * v.grad

## 5. モデル定義（ミニGPT）
計算量を抑えるため、教育向けの小さな設定を使います。

In [ ]:
# ===== ハイパーパラメータ =====
n_embd = 16
n_head = 4
n_layer = 1
block_size = 16
head_dim = n_embd // n_head

def matrix(nout, nin, std=0.08):
    return [[Value(random.gauss(0, std)) for _ in range(nin)] for _ in range(nout)]

state_dict = {
    'wte': matrix(vocab_size, n_embd),
    'wpe': matrix(block_size, n_embd),
    'lm_head': matrix(vocab_size, n_embd),
}

for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4 * n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embd, 4 * n_embd)

params = [p for mat in state_dict.values() for row in mat for p in row]
print(f"パラメータ数: {len(params)}")

In [ ]:
def linear(x, w):
    return [sum(wi * xi for wi, xi in zip(wo, x)) for wo in w]

def softmax(logits):
    max_val = max(val.data for val in logits)
    exps = [(val - max_val).exp() for val in logits]
    total = sum(exps)
    return [e / total for e in exps]

def rmsnorm(x):
    ms = sum(xi * xi for xi in x) / len(x)
    scale = (ms + 1e-5) ** -0.5
    return [xi * scale for xi in x]

def gpt(token_id, pos_id, keys, values):
    tok_emb = state_dict['wte'][token_id]
    pos_emb = state_dict['wpe'][pos_id]
    x = [t + p for t, p in zip(tok_emb, pos_emb)]
    x = rmsnorm(x)

    for li in range(n_layer):
        x_residual = x
        x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k)
        values[li].append(v)

        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            q_h = q[hs:hs + head_dim]
            k_h = [ki[hs:hs + head_dim] for ki in keys[li]]
            v_h = [vi[hs:hs + head_dim] for vi in values[li]]

            attn_logits = [
                sum(q_h[j] * k_h[t][j] for j in range(head_dim)) / head_dim ** 0.5
                for t in range(len(k_h))
            ]
            attn_weights = softmax(attn_logits)
            head_out = [
                sum(attn_weights[t] * v_h[t][j] for t in range(len(v_h)))
                for j in range(head_dim)
            ]
            x_attn.extend(head_out)

        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a + b for a, b in zip(x, x_residual)]

        x_residual = x
        x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a + b for a, b in zip(x, x_residual)]

    logits = linear(x, state_dict['lm_head'])
    return logits

## 6. 学習
学習ログを保存し、あとで損失曲線を描きます。

In [ ]:
learning_rate, beta1, beta2, eps_adam = 0.01, 0.85, 0.99, 1e-8
m = [0.0] * len(params)
v = [0.0] * len(params)

num_steps = 600
loss_history: List[float] = []

for step in range(num_steps):
    doc = docs[step % len(docs)]
    tokens = [BOS] + [stoi[ch] for ch in doc] + [BOS]
    n = min(block_size, len(tokens) - 1)

    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    losses = []
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id + 1]
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax(logits)
        loss_t = -probs[target_id].log()
        losses.append(loss_t)

    loss = (1 / n) * sum(losses)
    loss.backward()

    lr_t = learning_rate * (1 - step / num_steps)
    for i, p in enumerate(params):
        m[i] = beta1 * m[i] + (1 - beta1) * p.grad
        v[i] = beta2 * v[i] + (1 - beta2) * p.grad ** 2
        m_hat = m[i] / (1 - beta1 ** (step + 1))
        v_hat = v[i] / (1 - beta2 ** (step + 1))
        p.data -= lr_t * m_hat / (v_hat ** 0.5 + eps_adam)
        p.grad = 0

    loss_history.append(loss.data)

    if (step + 1) % 50 == 0:
        print(f"step {step+1:4d}/{num_steps} | loss={loss.data:.4f}")

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(loss_history, label='train loss')
plt.title('学習損失の推移')
plt.xlabel('step')
plt.ylabel('loss')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 7. 推論（新しい名前を生成）
温度 `temperature` を上げると多様性が増え、下げると安定します。

In [ ]:
def sample_one(temperature=0.8, max_len=block_size):
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    token_id = BOS
    out = []

    for pos_id in range(max_len):
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax([l / temperature for l in logits])
        token_id = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if token_id == BOS:
            break
        out.append(itos[token_id])

    return ''.join(out)

for i in range(12):
    print(f"sample {i+1:2d}: {sample_one(temperature=0.7)}")

## 8. 可視化: 次トークン確率 Top-10
先頭トークン（`<BOS>`）から1文字目として何が出やすいかを棒グラフで確認します。

In [ ]:
keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
logits = gpt(BOS, 0, keys, values)
probs = softmax(logits)
prob_values = [p.data for p in probs]

topk = 10
top_ids = sorted(range(vocab_size), key=lambda i: prob_values[i], reverse=True)[:topk]
top_labels = [itos[i] for i in top_ids]
top_probs = [prob_values[i] for i in top_ids]

plt.figure(figsize=(8, 4))
plt.bar(top_labels, top_probs)
plt.title('次トークン確率 Top-10 (開始トークンから)')
plt.xlabel('トークン')
plt.ylabel('確率')
plt.grid(axis='y', alpha=0.3)
plt.show()

for t, p in zip(top_labels, top_probs):
    print(f"{t:>5}: {p:.4f}")